In [ ]:
%store -r concat_df
%store -r data_avg
%store -r res

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import frameprocessing

In [ ]:

data_avg = data_avg.reset_index()
concat_df

#concat_df.loc[concat_df['Date'] == "2022-05-29"]
concat_df["unique string"] = concat_df.apply(lambda x: str(x["Date"]) + str(x["Home_team"]),axis=1)
concat_df = concat_df.drop_duplicates(subset = ["unique string"]).drop(["unique string"],axis=1)



#best_df = pd.DataFrame([])
#best_df.columns = concat_df.columns
#for row in concat_df:
#    if row["unique_string"] in 



In [ ]:
concat_df.loc[concat_df['Date'] == "2021-12-04"]


hot_data_avg = pd.get_dummies(data_avg["Team"])

avg_data_joined = data_avg.join(hot_data_avg)

data_avg_opponent = data_avg.copy(deep=True)
data_avg_opponent["Team_jord"] = frameprocessing.getCurrentTeamRows(data_avg_opponent)
hot_data_avg_opponent = pd.get_dummies(data_avg_opponent["Team_jord"])

avg_data_joined_opponent = data_avg_opponent.join(hot_data_avg_opponent)
avg_data_joined_opponent.drop(["Team_jord"],axis=1,inplace=True)
avg_data_joined_opponent

In [ ]:
def getCurrentTeamRows(df):
    temp = ['current_team '] * len(df['Team'])
    names = df['Team']

    current_columns = [y+x for x,y in zip(names,temp)]
    return current_columns





In [ ]:
def x_transform(df, avg_df):
    None


In [ ]:
concat_df = concat_df.loc[concat_df['Year']=="2021"]
concat_df.drop(["Year","Result"],axis=1,inplace=True)
concat_df = frameprocessing.one_hot_etc(concat_df)
X = concat_df.drop(['result_label'], axis=1)

y = concat_df['result_label']
#X = np.asarray(X).astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#vi vill att X_test ska ha hemmalag och datum
X_avg = frameprocessing.x_test_transform(X_test['Team'],X_test['opponent Team'],avg_df = avg_data_joined,opp_avg_df = avg_data_joined_opponent,home = X_test)
#X_test
#X_avg
#X_avg= frameprocessing.one_hot_etc(X_avg)
#X_test
X_test.drop(["opponent Team","Team"],axis=1,inplace=True)
X_avg.drop(["opponent Team","Team"],axis=1,inplace=True)
X_train.drop(["opponent Team","Team"],axis=1,inplace=True)
#concat_df


In [ ]:
X_testmannen = X_test[['Date','Home TEAM NAME']]
#X_train = frameprocessing.one_hot_etc(X_train)
#X_test = frameprocessing.one_hot_etc(X_test)
#X_test2 = X_test.copy(deep=True)

In [ ]:
encoder = LabelEncoder()

encoder.fit(y)
encoded_y_tjark = encoder.transform(y)
dummy = np_utils.to_categorical(encoded_y_tjark)

encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
dummy_y = np_utils.to_categorical(encoded_Y)

encoder.fit(y_test)
encoded_Y_test = encoder.transform(y_test)
dummy_y_test = np_utils.to_categorical(encoded_Y_test)


X_train = X_train.drop(['Home TEAM NAME',"Home_team","opponent Home_team","opponent Result","Date"],axis=1)
X_test = X_test.drop(['Home TEAM NAME',"Home_team","opponent Home_team","opponent Result","Date"],axis=1)
pd.set_option("display.max_columns", None)
#X_avg = X_avg.drop(['Home TEAM NAME',"Home_team","opponent Home_team","opponent Result","Date"],axis=1)



In [ ]:
X_train

In [ ]:
X_avg

In [ ]:
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
X_avg = np.asarray(X_avg).astype('float32')
model = Sequential()
model.add(Dense(len(X_train[0]), input_dim = len(X_train[0]), activation='relu'))
#model.add(Dense(64, input_dim=len(X_test.drop(['index'],axis = 1).columns), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, dummy_y, epochs=100, batch_size=100)

In [ ]:
#y_pred = model.predict(X_test2.drop(['index'],axis=1))

y_pred = model.predict(X_avg)#2.drop(['index'],axis=1))
print(np.around(y_pred, 2))
#-train_test_split
#-model predict
#-jämför med true värden från res


In [ ]:
X_testmannen["Date"] =  X_testmannen['Date'].apply(lambda x: x.replace("-","/"))
X_testmannen["Date"] = X_testmannen["Date"].apply(lambda x: frameprocessing.dateSwitcher(x))
print(X_testmannen.shape[0])
#X_testmannen
X_testmannen["Home TEAM NAME"] = X_testmannen["Home TEAM NAME"].apply(lambda x: frameprocessing.transformTeam(x))
test_df = X_testmannen.merge(res, how = "inner",left_on = ['Date',"Home TEAM NAME"],right_on=['Date','Home_team'])
test_df


y_true = test_df[["AvgH","AvgD","AvgA"]]
y_true = np.asarray(y_true).astype('float32')




In [ ]:
y_pred